In [1]:
import pandas as pd
train_data = pd.read_csv('train.csv', sep='\t', header=None)
test_data = pd.read_csv('test.csv', sep='\t', header=None)
train_data.columns = ["text","label"]
test_data.columns = ["text"]

In [2]:
test_data

,text
0,回放CCTV2的消费主张
1,给我打开玩具房的灯
2,循环播放赵本山的小品相亲来听
3,15号上午10点带孩子去海洋馆的行程帮我制定下。
4,把智能扫地机器人关掉
...,...
2995,是否能找一首2019年的抖音歌曲播放下呢
2996,下午三点有哪个台放大话西游吗帮我看下
2997,随机播放一首古筝弹奏的曲子可以吗
2998,美食纪录片螃蟹的征途给我找一下


In [3]:
shu_train = train_data.sample(frac=1)

In [4]:
label_2_id = {}
index = 0
for i in range(0,len(shu_train)):
    label = shu_train.iloc[i]["label"]
    if label not in label_2_id:
        label_2_id[label] = index
        index += 1

In [5]:
label_2_id

{'Other': 0,
 'Calendar-Query': 1,
 'Radio-Listen': 2,
 'TVProgram-Play': 3,
 'Weather-Query': 4,
 'Alarm-Update': 5,
 'HomeAppliance-Control': 6,
 'Video-Play': 7,
 'Music-Play': 8,
 'FilmTele-Play': 9,
 'Travel-Query': 10,
 'Audio-Play': 11}

In [28]:
index_2_label = {}
for label in label_2_id:
    index_2_label[label_2_id[label]] = label

In [29]:
index_2_label

{0: 'Other',
 1: 'Calendar-Query',
 2: 'Radio-Listen',
 3: 'TVProgram-Play',
 4: 'Weather-Query',
 5: 'Alarm-Update',
 6: 'HomeAppliance-Control',
 7: 'Video-Play',
 8: 'Music-Play',
 9: 'FilmTele-Play',
 10: 'Travel-Query',
 11: 'Audio-Play'}

In [76]:
import random
train_per = 0.8
train = []
valid = []
test = []
for i in range(0,int(len(shu_train)*train_per)):
    train.append((shu_train.iloc[i]["text"],label_2_id[shu_train.iloc[i]["label"]]))
for i in range(int(len(shu_train)*train_per),len(shu_train)):
    valid.append((shu_train.iloc[i]["text"],label_2_id[shu_train.iloc[i]["label"]]))
for i in range(0,len(test_data)):
    test.append((test_data.iloc[i]["text"],0))

In [77]:
test

[('回放CCTV2的消费主张', 0),
 ('给我打开玩具房的灯', 0),
 ('循环播放赵本山的小品相亲来听', 0),
 ('15号上午10点带孩子去海洋馆的行程帮我制定下。', 0),
 ('把智能扫地机器人关掉', 0),
 ('帮我续播16集摩天大楼', 0),
 ('放美国电影史密斯夫妇给我看', 0),
 ('放刘禹锡的浪淘沙来听', 0),
 ('查看6月6日是农历几月几号', 0),
 ('放一个讲述美食的美国纪录片来看', 0),
 ('周日上午10点左右陪妈妈逛街的行程提醒我一下。', 0),
 ('刘德华翻唱过的经典老歌都有哪些，尤其是70年代的', 0),
 ('看一下怎么去仙游有汽车还是火车那个快', 0),
 ('非常6+1-2021-20节目想回看一下', 0),
 ('收听频率是九七点四的中国歌曲排行榜', 0),
 ('明天下午的备忘录请给我全部关闭，不要让它再提醒', 0),
 ('王力宏在重庆开演唱会的视频放来看一下', 0),
 ('明天紫外线强不强', 0),
 ('今儿这风是5级还是7级', 0),
 ('发表感言章子怡数次哽咽', 0),
 ('我想看5月9日兹维列夫对蒂姆的网球比赛视频', 0),
 ('给我放推理剧的天道可以吗', 0),
 ('钱塘江最近会不会下大暴雨', 0),
 ('提醒我海南气温升高', 0),
 ('刚被联合国封为地球卫士的周迅穿得很低碳', 0),
 ('我不想睡觉我想聊天说话', 0),
 ('我想看今天的体育台播了啥，帮我找一下', 0),
 ('我想看2017年李荣浩和杨丞琳在演唱会上演唱的年轮说的视频', 0),
 ('我想听邵帅唱的驻马店', 0),
 ('现在给我建一个日程下午2点去邮局买邮票', 0),
 ('没想到的是严立恒实则与夏友善是同父异母的姐弟', 0),
 ('我把赵英俊的送你一朵小红花下载下来循环播放', 0),
 ('音乐台的话给我找河南音乐广播', 0),
 ('播放邓紫棋唱歌的视频', 0),
 ('看一下北京文艺广播的今晚我们说电影还在播不', 0),
 ('我想看单依纯在为歌而赞唱的如此的纯享版音乐视频', 0),
 ('我超级想看2020年张大仙王者荣耀录播的游戏视频', 0),
 ('给我找一下中国电视剧觉醒年代的拍戏花絮', 0),
 (

In [7]:
import numpy as np
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Lambda, Dense

Using TensorFlow backend.


In [8]:
set_gelu('tanh')

In [84]:
num_classes = 12
maxlen = 128
batch_size = 32
config_path = 'G:\\deep_learning\\models\\chinese_roberta_wwm_ext_L-12_H-768_A-12\\bert_config.json'
checkpoint_path = 'G:\\deep_learning\\models\\chinese_roberta_wwm_ext_L-12_H-768_A-12\\bert_model.ckpt'
dict_path = 'G:\\deep_learning\\models\\chinese_roberta_wwm_ext_L-12_H-768_A-12\\vocab.txt'

In [85]:
# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

In [86]:
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

In [87]:
# 加载预训练模型
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='bert',
    return_keras_model=False,
)

In [88]:
output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.model.output)
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
___________________________________________________________________________________________

In [89]:
AdamLR = extend_with_piecewise_linear_lr(Adam, name='AdamLR')
AdamLR = extend_with_piecewise_linear_lr(Adam, name='AdamLR')

model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=AdamLR(learning_rate=1e-4, lr_schedule={
        1000: 1,
        2000: 0.1
    }),
    metrics=['accuracy'],
)

In [90]:
train_generator = data_generator(train, batch_size)
valid_generator = data_generator(valid, batch_size)
test_generator = data_generator(test, batch_size)

In [91]:
from sklearn.metrics import f1_score

In [92]:
def evaluate(data):
    targets, preds = [],[]
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        preds.extend(y_pred.tolist())
        y_true = y_true[:, 0]
        targets.extend(y_true.tolist())
    val_f1 = f1_score(targets, preds, average='micro')
    return val_f1

In [93]:
class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_f1 = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_f1 = evaluate(valid_generator)
        if val_f1 > self.best_val_f1:
            self.best_val_f1 = val_f1
            model.save_weights('G:\\deep_learning\\Coggle\\202303\\results\\roberta\\best_model.weights')
        print(
            u'val_f1: %.5f, best_val_f1: %.5f' %
            (val_f1, self.best_val_f1)
        )

In [94]:
evaluator = Evaluator()

In [95]:
model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=5,
        callbacks=[evaluator]
    )

Epoch 1/5
303/303 [==============================] - 47s 157ms/step - loss: 0.8674 - accuracy: 0.7656
val_f1: 0.94008, best_val_f1: 0.94008
Epoch 2/5
303/303 [==============================] - 39s 127ms/step - loss: 0.1974 - accuracy: 0.9416
val_f1: 0.94256, best_val_f1: 0.94256
Epoch 3/5
303/303 [==============================] - 39s 128ms/step - loss: 0.1506 - accuracy: 0.9556
val_f1: 0.92727, best_val_f1: 0.94256
Epoch 4/5
303/303 [==============================] - 39s 128ms/step - loss: 0.1338 - accuracy: 0.9583
val_f1: 0.94793, best_val_f1: 0.94793
Epoch 5/5
303/303 [==============================] - 39s 128ms/step - loss: 0.0640 - accuracy: 0.9810
val_f1: 0.94959, best_val_f1: 0.94959


In [96]:
def _test(data):
    preds = []
    for x_true, _ in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        preds.extend(y_pred.tolist())
    return preds

In [97]:
test_preds = _test(test_generator)

In [98]:
test_data["pred_by_roberta"] = test_preds

In [99]:
test_data

,text,pred_by_roberta
0,回放CCTV2的消费主张,3
1,给我打开玩具房的灯,6
2,循环播放赵本山的小品相亲来听,11
3,15号上午10点带孩子去海洋馆的行程帮我制定下。,5
4,把智能扫地机器人关掉,6
...,...,...
2995,是否能找一首2019年的抖音歌曲播放下呢,8
2996,下午三点有哪个台放大话西游吗帮我看下,3
2997,随机播放一首古筝弹奏的曲子可以吗,8
2998,美食纪录片螃蟹的征途给我找一下,7


In [35]:
#在https://competition.coggle.club/上的结果是0.8340
#这个是使用开源roberta,max_len=50,batch_size=32,epoch=5
with open("results\\roberta_50_32_5.txt","w") as f:
    f.write("ID,Target\n")
    for i in range(len(test_data)):
        f.write(str(i+1)+","+index_2_label[test_data.iloc[i]["pred_by_roberta"]]+"\n")

In [83]:
#在https://competition.coggle.club/上的结果是0.830333
#这个是使用开源roberta,max_len=32,batch_size=32,epoch=5
with open("results\\roberta_32_32_5.txt","w") as f:
    f.write("ID,Target\n")
    for i in range(len(test_data)):
        f.write(str(i+1)+","+index_2_label[test_data.iloc[i]["pred_by_roberta"]]+"\n")

In [100]:
#在https://competition.coggle.club/上的结果是0.839667
#这个是使用开源roberta,max_len=128,batch_size=32,epoch=5
with open("results\\roberta_128_32_5.txt","w") as f:
    f.write("ID,Target\n")
    for i in range(len(test_data)):
        f.write(str(i+1)+","+index_2_label[test_data.iloc[i]["pred_by_roberta"]]+"\n")

In [101]:
#BERT模型精度与文本最大长度是否相关？
#答:与文本最大长度相关，在正常范围内增长max length有助于精度提升

In [1]:
#BERT模型分类时最后全连接层的输入是什么含义？
#以下答案by ChatGPT:
'''
BERT模型分类时最后全连接层的输入是BERT模型输出的第一个token的向量表示，也就是CLS标记的向量表示。

在BERT模型中，输入序列的第一个token是一个特殊的标记CLS（classification），它被用于分类任务。在模型训练时，BERT模型会对整个输入序列进行处理，并在序列的每个位置生成一个对应的输出向量。其中，CLS标记对应的输出向量被用作整个序列的表示，传递给分类器进行分类。

在进行分类任务时，我们可以使用CLS标记的向量表示来表示整个句子的语义信息，然后将其输入到一个全连接层中进行分类。因为该向量包含整个句子的信息，所以它可以有效地捕捉句子的语义信息，并且可以很好地适应不同长度的输入。
'''

'\nBERT模型分类时最后全连接层的输入是BERT模型输出的第一个token的向量表示，也就是CLS标记的向量表示。\n\n在BERT模型中，输入序列的第一个token是一个特殊的标记CLS（classification），它被用于分类任务。在模型训练时，BERT模型会对整个输入序列进行处理，并在序列的每个位置生成一个对应的输出向量。其中，CLS标记对应的输出向量被用作整个序列的表示，传递给分类器进行分类。\n\n在进行分类任务时，我们可以使用CLS标记的向量表示来表示整个句子的语义信息，然后将其输入到一个全连接层中进行分类。因为该向量包含整个句子的信息，所以它可以有效地捕捉句子的语义信息，并且可以很好地适应不同长度的输入。\n'